In [1]:
import os

# os.environ["WANDB_API_KEY"] = '+++++++++++'  # 将引号内的+替换成自己在wandb上的一串值
# os.environ["WANDB_MODE"] = "offline"  # 离线  （此行代码不用修改）

import json

import pandas as pd
import torch
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from swanlab.integration.huggingface import SwanLabCallback
from transformers import DataCollatorForSeq2Seq, Trainer
import os

/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
glm4_model_path = '/home/LLM_para/para_Qwen'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 加载model和tokenizer
tokenizer = AutoTokenizer.from_pretrained(glm4_model_path, use_fast=False,
                                          trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(glm4_model_path,
                                             #quantization_config=bnb_config,
                                             device_map="auto", torch_dtype=torch.bfloat16,
#                                              attn_implementation="flash_attention_2",
                                             trust_remote_code=True)
model.config.use_cache = False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [01:56<00:00, 29.11s/it]


In [4]:
model.enable_input_require_grads()  #  开启梯度检查点
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
#结束标识符
print(tokenizer.eos_token)
tokenizer.encode('<|endoftext|>')

<|endoftext|>


[151643]

In [5]:
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import re
#  1600 8   2200 6   2600 
max_token = 1100
#  保证特殊字符能被识别
accept_toekn = max_token - 170
#获取REF表数据
tab_str = ""
schema_slicing = []
df = pd.read_csv("./mydataset_new/table_schema_Reference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"

    
print(len(schema_slicing))

# print(ReF_group[5])
# print("#############################")
# print(ReF_group[6])


#获取noREF表数据
df = pd.read_csv("./mydataset_new/table_schema_noReference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
#         print("+++++++++++")
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"
        
        
if len(tab_str) > 0:
    schema_slicing.append(tab_str)

print(len(schema_slicing))
# print(schema_slicing[6])
# print("#############################")
# print(schema_slicing[7])
# print(schema_slicing[8])

create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
for piece in schema_slicing:
    create_table_statements = create_table_pattern.findall(piece)
    temp_list = []
    for table in create_table_statements:
        table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
        temp_list.append(table_name.lower())
        test_total.add(table_name.lower())
        count+=1
    print(temp_list)
    table_name_group.append(temp_list)
# print(count)
print(len(test_total))
#     table_name_group.append()

100%|████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 422.56it/s]


6


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 440.66it/s]

8
['manufacturers', 'products', 'student', 'plays_games', 'sportsinfo', 'actor', 'entrepreneur', 'people', 'ref_hotel_star_ratings', 'ref_attraction_types', 'hotels', 'tourist_attractions', 'street_markets', 'shops', 'museums', 'royal_family', 'theme_parks', 'visits', 'photos', 'staff']
['tourist_attraction_features', 'wrestler', 'elimination', 'business', 'category', 'checkin', 'neighbourhood', 'review', 'tip', 'ref_detention_type', 'ref_incident_type', 'addresses', 'students', 'teachers', 'assessment_notes', 'behavior_incident', 'detention']
['student_addresses', 'students_in_detention', 'film', 'film_market_estimation', 'catalogs', 'catalog_structure', 'catalog_contents', 'catalog_contents_additional_attributes', 'routes', 'airports', 'stadium', 'game', 'injury_accident', 'physician', 'department', 'affiliated_with', 'trained_in', 'patient', 'nurse']
['appointment', 'prescribes', 'block', 'room', 'on_call', 'stay', 'undergoes', 'buildings', 'office_locations', 'region', 'party', 'me

In [6]:
def fill_dataset_template(schema_pad,question_pad,selected_table_pad,answer_pad):
     message = {
                "instruction": f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema_pad}",
                "input": f"{question_pad}######selected_table: {selected_table_pad}.",
                "output": f"```Reference Table\n-- Tables: {answer_pad};\n```",
            }
     return message
#  #None#

In [7]:
df = pd.read_csv("./mydataset_new/full_finetuning_dataset_cropped.csv",encoding="utf-8")
outer_index = 0
message_group = []
for i in range(len(df['db_id'])):
    question = df['question'][i]
    query = df['query'][i]
    correct_table = df['correct_tables'][i]
    correct_table_list = correct_table.split(", ")
    dict_target_table_slice = {}
    for target_table in correct_table_list:
        for index, table_name_slicing in enumerate(table_name_group): 
            if target_table.lower() in table_name_slicing:
                dict_target_table_slice[target_table] = index
                break    
#     if  len(dict_target_table_slice) != len(correct_table_list):
#         print(f"error--{outer_index}----",list(dict_target_table_slice.keys()),"=====",correct_table_list)
#     outer_index += 1     
#     基于定位构建两条数据集: 一条顺序 一条判断
#     顺序
    sorted_relation = sorted(dict_target_table_slice.items(), key=lambda x: x[1])
    selected_table_list = []
    for index, piece in enumerate(schema_slicing):
        this_selected = []
        for key, value in sorted_relation:
            if index == value:
                this_selected.append(key)
#          给与选择提示
        message = None
        if len(selected_table_list) > 0:
            selected_table_str = ', '.join(selected_table_list)
            if len(this_selected) == 0:
                message = fill_dataset_template(piece,question,selected_table_str,"#None#")
            else:
                this_selected_str = ', '.join(this_selected)
                message = fill_dataset_template(piece,question,selected_table_str,this_selected_str)
        if message is not None:
            message_group.append(message)
#          不给与选择提示 + 顺序决策   
        if len(this_selected) == 0:
            message = fill_dataset_template(piece,question,"None","#None#")
        else:
            this_selected_str = ', '.join(this_selected)
            message = fill_dataset_template(piece,question,"None",this_selected_str)
        
        if len(this_selected) > 0:
            for item in this_selected:
                selected_table_list.append(item)
                
        message_group.append(message)
        
with open("T2Q_Qwen_SFT_train_table_myidea.jsonl", "w", encoding="utf-8") as file:
    for message in message_group:
        file.write(json.dumps(message, ensure_ascii=False) + "\n")

print(len(message_group))
#     判断

        
#     table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
#     print(correct_table_list)
    #      schema_slicing
#     db_id = df['db_id'][i].replace(" ", "")
#     message = {
#         "instruction": f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{database_schema}",
#         "input": f"{question}",
#         "output": f"```Reference Table\n-- Tables: {correct_tables};\n```",
#     }
#     messages.append(message)
#     # print(message["instruction"])
#     # print(message["input"])
#     # print(message["output"])
#     print("++++++++++++++++++++++++++++++++++++++++++++++++++++")
#     if max_length < len(message["instruction"]) + len(message["input"]) + len(message["output"]):
#         max_length = len(message["instruction"]) + len(message["input"]) + len(message["output"])
#         max_ins = message["instruction"]
#         max_in = message["input"]
#         max_out = message["output"]

# with open("./My_idea_dataset/T2Q_GLM4_SFT_train_table.jsonl", "w", encoding="utf-8") as file:
#     for message in messages:
#         file.write(json.dumps(message, ensure_ascii=False) + "\n")

20087


In [9]:
# 生成训练集
process_path = "T2Q_Qwen_SFT_train_table_myidea.jsonl"
max_len = 0
count = 0
with open(process_path, "r") as file:
        for line in file:
            count+=1
            # 解析每一行的json数据
            example = json.loads(line)
            if count == 1:
                print( f"<|im_start|>system\n{example['instruction']}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n")
            print(f"index:---------{count}")
            instruction = tokenizer(
         f"<|im_start|>system\n{example['instruction']}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False, )
            response = tokenizer(f"{example['output']}<|endoftext|>\n", add_special_tokens=False)
            input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
            attention_mask = (
            instruction["attention_mask"] + response["attention_mask"] + [1])
            labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
            if len(input_ids) > max_len:
                max_len = len(input_ids)
print("max_len:",max_len)   
#1860

<|im_start|>system
 I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. 
##instruction:CREATE TABLE `Manufacturers` (
  Code INTEGER,
  Name VARCHAR(255),
  Headquarter VARCHAR(255),
  Founder VARCHAR(255),
  Revenue REAL
);
CREATE TABLE `Products` (
  Code INTEGER,
  Name VARCHAR(255),
  Price DECIMAL,
  Manufacturer INTEGER REFERENCES Manufacturers(Code)
);
CREATE TABLE `Student` (
  StuID INTEGER,
  LName VARCHAR(12),
  Fname VARCHAR(12),
  Age INTEGER,
  Sex VARCHAR(1),
  Major INTEGER,
  Advisor INTEGER,
  city_code VARCHAR(3)
);
CREATE TABLE `Plays_Games` (
  StuID INTEGER REFERENCES Student(StuID),
  GameID INTEGER REFERENCES Video_Games(GameID),
  Hours_Played INTEGER
);
CREATE TABLE `SportsInfo` (
  StuID INTEGER REFERENCES Student(StuID),
  SportName VA

index:---------264
index:---------265
index:---------266
index:---------267
index:---------268
index:---------269
index:---------270
index:---------271
index:---------272
index:---------273
index:---------274
index:---------275
index:---------276
index:---------277
index:---------278
index:---------279
index:---------280
index:---------281
index:---------282
index:---------283
index:---------284
index:---------285
index:---------286
index:---------287
index:---------288
index:---------289
index:---------290
index:---------291
index:---------292
index:---------293
index:---------294
index:---------295
index:---------296
index:---------297
index:---------298
index:---------299
index:---------300
index:---------301
index:---------302
index:---------303
index:---------304
index:---------305
index:---------306
index:---------307
index:---------308
index:---------309
index:---------310
index:---------311
index:---------312
index:---------313
index:---------314
index:---------315
index:------

index:---------759
index:---------760
index:---------761
index:---------762
index:---------763
index:---------764
index:---------765
index:---------766
index:---------767
index:---------768
index:---------769
index:---------770
index:---------771
index:---------772
index:---------773
index:---------774
index:---------775
index:---------776
index:---------777
index:---------778
index:---------779
index:---------780
index:---------781
index:---------782
index:---------783
index:---------784
index:---------785
index:---------786
index:---------787
index:---------788
index:---------789
index:---------790
index:---------791
index:---------792
index:---------793
index:---------794
index:---------795
index:---------796
index:---------797
index:---------798
index:---------799
index:---------800
index:---------801
index:---------802
index:---------803
index:---------804
index:---------805
index:---------806
index:---------807
index:---------808
index:---------809
index:---------810
index:------

index:---------1233
index:---------1234
index:---------1235
index:---------1236
index:---------1237
index:---------1238
index:---------1239
index:---------1240
index:---------1241
index:---------1242
index:---------1243
index:---------1244
index:---------1245
index:---------1246
index:---------1247
index:---------1248
index:---------1249
index:---------1250
index:---------1251
index:---------1252
index:---------1253
index:---------1254
index:---------1255
index:---------1256
index:---------1257
index:---------1258
index:---------1259
index:---------1260
index:---------1261
index:---------1262
index:---------1263
index:---------1264
index:---------1265
index:---------1266
index:---------1267
index:---------1268
index:---------1269
index:---------1270
index:---------1271
index:---------1272
index:---------1273
index:---------1274
index:---------1275
index:---------1276
index:---------1277
index:---------1278
index:---------1279
index:---------1280
index:---------1281
index:---------1282


index:---------1661
index:---------1662
index:---------1663
index:---------1664
index:---------1665
index:---------1666
index:---------1667
index:---------1668
index:---------1669
index:---------1670
index:---------1671
index:---------1672
index:---------1673
index:---------1674
index:---------1675
index:---------1676
index:---------1677
index:---------1678
index:---------1679
index:---------1680
index:---------1681
index:---------1682
index:---------1683
index:---------1684
index:---------1685
index:---------1686
index:---------1687
index:---------1688
index:---------1689
index:---------1690
index:---------1691
index:---------1692
index:---------1693
index:---------1694
index:---------1695
index:---------1696
index:---------1697
index:---------1698
index:---------1699
index:---------1700
index:---------1701
index:---------1702
index:---------1703
index:---------1704
index:---------1705
index:---------1706
index:---------1707
index:---------1708
index:---------1709
index:---------1710


...

index:---------24233
index:---------24234
index:---------24235
index:---------24236
index:---------24237
index:---------24238
index:---------24239
index:---------24240
index:---------24241
index:---------24242
index:---------24243
index:---------24244
index:---------24245
index:---------24246
index:---------24247
index:---------24248
index:---------24249
index:---------24250
index:---------24251
index:---------24252
index:---------24253
index:---------24254
index:---------24255
index:---------24256
index:---------24257
index:---------24258
index:---------24259
index:---------24260
index:---------24261
index:---------24262
index:---------24263
index:---------24264
index:---------24265
index:---------24266
index:---------24267
index:---------24268
index:---------24269
index:---------24270
index:---------24271
index:---------24272
index:---------24273
index:---------24274
index:---------24275
index:---------24276
index:---------24277
index:---------24278
index:---------24279
index:-------

index:---------24689
index:---------24690
index:---------24691
index:---------24692
index:---------24693
index:---------24694
index:---------24695
index:---------24696
index:---------24697
index:---------24698
index:---------24699
index:---------24700
index:---------24701
index:---------24702
index:---------24703
index:---------24704
index:---------24705
index:---------24706
index:---------24707
index:---------24708
index:---------24709
index:---------24710
index:---------24711
index:---------24712
index:---------24713
index:---------24714
index:---------24715
index:---------24716
index:---------24717
index:---------24718
index:---------24719
index:---------24720
index:---------24721
index:---------24722
index:---------24723
index:---------24724
index:---------24725
index:---------24726
index:---------24727
index:---------24728
index:---------24729
index:---------24730
index:---------24731
index:---------24732
index:---------24733
index:---------24734
index:---------24735
index:-------

index:---------25126
index:---------25127
index:---------25128
index:---------25129
index:---------25130
index:---------25131
index:---------25132
index:---------25133
index:---------25134
index:---------25135
index:---------25136
index:---------25137
index:---------25138
index:---------25139
index:---------25140
index:---------25141
index:---------25142
index:---------25143
index:---------25144
index:---------25145
index:---------25146
index:---------25147
index:---------25148
index:---------25149
index:---------25150
index:---------25151
index:---------25152
index:---------25153
index:---------25154
index:---------25155
index:---------25156
index:---------25157
index:---------25158
index:---------25159
index:---------25160
index:---------25161
index:---------25162
index:---------25163
index:---------25164
index:---------25165
index:---------25166
index:---------25167
index:---------25168
index:---------25169
index:---------25170
index:---------25171
index:---------25172
index:-------

index:---------25563
index:---------25564
index:---------25565
index:---------25566
index:---------25567
index:---------25568
index:---------25569
index:---------25570
index:---------25571
index:---------25572
index:---------25573
index:---------25574
index:---------25575
index:---------25576
index:---------25577
index:---------25578
index:---------25579
index:---------25580
index:---------25581
index:---------25582
index:---------25583
index:---------25584
index:---------25585
index:---------25586
index:---------25587
index:---------25588
index:---------25589
index:---------25590
index:---------25591
index:---------25592
index:---------25593
index:---------25594
index:---------25595
index:---------25596
index:---------25597
index:---------25598
index:---------25599
index:---------25600
index:---------25601
index:---------25602
index:---------25603
index:---------25604
index:---------25605
index:---------25606
index:---------25607
index:---------25608
index:---------25609
index:-------

index:---------25996
index:---------25997
index:---------25998
index:---------25999
index:---------26000
index:---------26001
index:---------26002
index:---------26003
index:---------26004
index:---------26005
index:---------26006
index:---------26007
index:---------26008
index:---------26009
index:---------26010
index:---------26011
index:---------26012
index:---------26013
index:---------26014
index:---------26015
index:---------26016
index:---------26017
index:---------26018
index:---------26019
index:---------26020
index:---------26021
index:---------26022
index:---------26023
index:---------26024
index:---------26025
index:---------26026
index:---------26027
index:---------26028
index:---------26029
index:---------26030
index:---------26031
index:---------26032
index:---------26033
index:---------26034
index:---------26035
index:---------26036
index:---------26037
index:---------26038
index:---------26039
index:---------26040
index:---------26041
index:---------26042
index:-------

In [8]:
#  和下面的max_len 作一个合并
#  max_seq_length = 2100
def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = max_token 
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n{example['instruction']}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False, )
   
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = (
        instruction["attention_mask"] + response["attention_mask"] + [1]
    )
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [9]:
#  set dataset
train_path = "T2Q_Qwen_SFT_train_table_myidea.jsonl"
# val_path = "/home/code/chat_SQL/Dataset/GLM4/ENG_ori/My_idea_dataset/T2Q_GLM4_SFT_val_table.jsonl"

train_df = pd.read_json(train_path, lines=True)
train_ds = Dataset.from_pandas(train_df)
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)


# val_df = pd.read_json(val_path, lines=True)
# val_ds = Dataset.from_pandas(val_df)
# val_dataset = val_ds.map(process_func, remove_columns=val_ds.column_names)

Map: 100%|████████████████████████████████████████████████████████████████| 20087/20087 [01:18<00:00, 256.59 examples/s]


In [10]:
#  train_config
#  示例里 r 为 8
lora_r = 64
lora_alpha = 32
lora_dropout = 0.1
output_dir = "./Qwen2_SFT_Table_1100T_3Epoch"
num_train_epochs = 3
bf16 = True
overwrite_output_dir = True
per_device_train_batch_size = 2
per_device_eval_batch_size = 20
gradient_accumulation_steps = 16
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 5e-5
weight_decay = 0.01
lr_scheduler_type = "cosine"
warmup_ratio = 0.01
max_grad_norm = 0.3
group_by_length = True
auto_find_batch_size = False
save_steps = 50
logging_steps = 50
load_best_model_at_end= False
packing = False
save_total_limit=4
neftune_noise_alpha=5
# report_to="wandb"
max_seq_length = max_token

In [11]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
       'up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj'
    ],
    task_type=TaskType.CAUSAL_LM,
)

In [12]:
swanlab_callback = SwanLabCallback(
    project="LLAMA3-SFT_T2QTab",
    experiment_name="GLM4-9B-Chat",
    description="LLAMA3模型在spider_CN数据集上微调。",
    config={
        "model": "/home/LLM_para/LLAMA3",
        "dataset": "my_new_dataset",
    },
)

In [13]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
#     evaluation_strategy=evaluation_strategy,
    max_grad_norm = max_grad_norm,
    auto_find_batch_size = auto_find_batch_size,
    save_total_limit = save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="none",
    neftune_noise_alpha= neftune_noise_alpha
)

In [14]:
# response_template = "### Response:"
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
# collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset['train'],
#     eval_dataset=dataset['validation'],
#     peft_config=peft_config,
#     data_collator=collator,
#     args=training_arguments,
#     max_seq_length=max_seq_length,
#     packing=packing
# )
model = get_peft_model(model, peft_config)
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
#     eval_dataset= val_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
# import os
# os.environ['CUDA_HOME'] = '/usr/local/cuda'
trainer.train()

swanlab: Tracking run with swanlab version 0.3.13                                  
swanlab: Run data will be saved locally in /home/code/chat_SQL/main_verify/Qwen_crop/swanlog/run-20240902_123945-a3b1799d
swanlab: 👋 Hi winhong, welcome to swanlab!
swanlab: Syncing run GLM4-9B-Chat_Sep02_12-39-45 to the cloud
swanlab: 🌟 Run `swanlab watch -l /home/code/chat_SQL/main_verify/Qwen_crop/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@winhong/LLAMA3-SFT_T2QTab
swanlab: 🚀 View run at https://swanlab.cn/@winhong/LLAMA3-SFT_T2QTab/runs/lbokb6lt5plwo44581mty


Step,Training Loss
50,2.117600
100,0.019800
150,0.016900
200,0.013000
250,0.012300
300,0.012600
350,0.010300
400,0.009400
450,0.009600
500,0.008300


/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_Qwen - will assume that the vocabulary was not modified.
  warnings.warn(
/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_Qwen - will assume that the vocabulary was not modified.
  warnings.warn(
/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_Qwen - will assume that the vocabulary was not modified.
  warnings.warn(
/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_Qwen - will assume that the vocabulary was not modified.
  warnings.warn(
/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-p

/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_Qwen - will assume that the vocabulary was not modified.
  warnings.warn(
/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_Qwen - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=1881, training_loss=0.06139763392575366, metrics={'train_runtime': 14710.4965, 'train_samples_per_second': 4.096, 'train_steps_per_second': 0.128, 'total_flos': 2.6409272542194125e+18, 'train_loss': 0.06139763392575366, 'epoch': 2.996415770609319})

In [16]:
output_dir = os.path.join("./", "final_checkpoint_Qwen2_TN_1100T_3Epoch")
trainer.model.save_pretrained(output_dir)

/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_Qwen - will assume that the vocabulary was not modified.
  warnings.warn(
